# 建立ResNet-34对花朵图片进行分类
对残差块也建立类，并通过``model.add_module(name, module)``的方式向model中添加子model

使用GPU进行训练

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision import transforms
import matplotlib.pyplot as plt

cuda_gpu = torch.cuda.is_available()
if cuda_gpu:
    torch.cuda.set_device(1)
    print(torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('There is no available gpu.')

There is no available gpu.


In [2]:
data_dir = os.path.join('..', 'data')
flower_dir = os.path.join(data_dir, 'flower_photos')
train_dir = os.path.join(flower_dir, 'train')
test_dir = os.path.join(flower_dir, 'test')

## transforms, ImageFolder, DataLoader
* 使用``torchvision.transforms.Compose``定义一些Data augmentation操作
* 首先使用``torchvision.datasets.ImageFolder``读取出图片数据，之后传入transforms操作，对图片进行处理
* 使用``torch.utils.data.DataLoader``定义Dataloader

### transforms.ToTensor()
将PIL Image或者 ndarray 转换为tensor，并且归一化至$[0-1]$
* 注意事项：归一化至$[0-1]$是直接除以255，若自己的ndarray数据尺度有变化，则需要自行修改。

### transforms.RandomRotation(degrees, resample=False, expand=False, center=None)
依degrees随机旋转一定角度

参数：
* ``degress``- (sequence or float or int) ，若为单个数，如 30，则表示在（-30，+30）之间随机旋转
若为sequence，如(30，60)，则表示在30-60度之间随机旋转
* ``resample``- 重采样方法选择，可选 PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC，默认为最近邻
* ``expand``- Optional expansion flag. If true, expands the output to make it large enough to hold the entire rotated image. If false or omitted, make the output image the same size as the input image. Note that the expand flag assumes rotation around the center and no translation.
* ``center``- 可选为中心旋转还是左上角旋转。Default is the center of the image.

**需要注意，``transforms.Resize(size)``如果只传入一个int，则会将尺寸为(height, width)的图片变为(size * height / width, size)(height > width)。**

In [3]:
input_size = 224
batch_size = 64

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor()
    ]),
    "val": transforms.Compose([
        transforms.Resize((input_size, input_size)),
#         transforms.CenterCrop(input_size),
        transforms.ToTensor()
    ])
}

image_datasets = {x: ImageFolder(os.path.join(flower_dir, x), data_transforms[x]) for x in ["train", "val"]}

train_loader, test_loader = [torch.utils.data.DataLoader(image_datasets[x], 
        batch_size=batch_size, shuffle=True, num_workers=4) for x in ["train", "val"]]

## 建立残差模块
其中每个残差模块的第一个残差快可能需要使用$1\times 1$的卷积改变通道数

In [4]:
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        h = self.relu(self.bn1(self.conv1(x)))
        h = self.bn2(self.conv2(h))
        if self.conv3:
            x = self.conv3(x)
        return self.relu(h + x)

## 建立ResNet类
首先将残差模块之前的部分定义为``self.model``，并将每个残差模块通过``model.add_module()``的方式加入到``self.model``中。

### add_module(name, module)
Adds a child module to the current module.The module can be accessed as an attribute using the given name.

**Parameters:**
* ``name (string)`` – name of the child module. The child module can be accessed from this module using the given name
* ``module (Module)`` – child module to be added to the module.

In [5]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        
        def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
            if first_block:
                assert in_channels == out_channels # 第一个模块的通道数同输入通道数一致
            layers = []
            for i in range(num_residuals):
                # 除了第一个残差模块外的所有残差模块的第一个残差块都需要通过1x1卷积改变通道数
                if i == 0 and not first_block:
                    layers.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
                else:
                    layers.append(Residual(out_channels, out_channels))
            return nn.Sequential(*layers)
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.model.add_module("resnet_block1", resnet_block(64, 64, 3, first_block=True))
        self.model.add_module("resnet_block2", resnet_block(64, 128, 4))
        self.model.add_module("resnet_block3", resnet_block(128, 256, 6))
        self.model.add_module("resnet_block4", resnet_block(256, 512, 3))
        
        self.avg_pool = nn.AvgPool2d(7)
        self.linear = nn.Linear(512, 5)
        
    def forward(self, x):
        h = self.model(x)
        h = self.avg_pool(h)
        h = h.view(-1, 512)
        h = self.linear(h)
        return h


if cuda_gpu:
    net = ResNet().cuda()
else:
    net = ResNet()

### 查看网络结构

In [6]:
print(net)

ResNet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (resnet_block1): Sequential(
      (0): Residual(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): Residual(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [5]:
#nn.CrossEntropyLoss()中已包含softmax激活运算
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

### 如使用GPU，则需要将数据也转为cuda，并且令标签数据和预测结果类型相同（同为cuda或同为cpu）

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    train_correct = 0
    train_total = 0
    train_loss = 0.
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        
        # 将数据转为cuda
        if cuda_gpu:
            inputs = inputs.cuda()
#             labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        
        # 将输出变回cpu，如果之前将labels转为cuda，这里就可以省略，需要保证outputs和labels类型相同
        if cuda_gpu:
            outputs = outputs.cpu()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        value_pred, label_pred = torch.max(outputs, axis=1)
        train_correct += (labels == label_pred).sum().item()
        train_total += labels.shape[0]
        train_loss += loss.item() * labels.shape[0]

    train_loss /= train_total
    train_correct /= train_total

    # print statistics
    print('Train Epoch: %d\nTrain: Loss: %.4f, accuracy: %.4f ' % (epoch, train_loss, train_correct), end='')

    test_correct = 0
    test_total = 0
    test_loss = 0.
    with torch.no_grad():
        for images, labels in test_loader:
            if cuda_gpu:
                images = images.cuda()
                labels = labels.cuda()
            
            y_pred = net(images)
            value_pred, label_pred = torch.max(y_pred, axis=1)
            test_correct += (labels == label_pred).sum().item()
            test_total += labels.shape[0]
            loss_batch = criterion(y_pred, labels)
            test_loss += loss_batch.item() * labels.shape[0]

        test_loss /= test_total
        test_correct /= test_total
        print('Test: Loss: %.4f, accuracy: %.4f' % (test_loss, test_correct))

print('Finished Training')